In [27]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [10]:
train_dir = '../Images/Class/'
test_dir = '../Images/output_images/'


In [11]:
image_size = (224, 224)
batch_size = 32


In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [13]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 152 images belonging to 5 classes.


In [14]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5 images belonging to 1 classes.


In [15]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(1024, 1024, 3))


In [16]:
for layer in base_model.layers:
    layer.trainable = False
    

In [17]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())


In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))


In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 32, 32, 2048)      23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                        

In [21]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
5/5 [==============================] - 6s 999ms/step - loss: 1.6365 - accuracy: 0.1645 - val_loss: 8.0813 - val_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 4s 867ms/step - loss: 1.6193 - accuracy: 0.1842 - val_loss: 8.0638 - val_accuracy: 0.0000e+00
Epoch 3/10
5/5 [==============================] - 4s 819ms/step - loss: 1.6315 - accuracy: 0.2171 - val_loss: 8.1173 - val_accuracy: 0.0000e+00
Epoch 4/10
5/5 [==============================] - 5s 918ms/step - loss: 1.6128 - accuracy: 0.1842 - val_loss: 8.0638 - val_accuracy: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 5s 894ms/step - loss: 1.6221 - accuracy: 0.1776 - val_loss: 8.0621 - val_accuracy: 0.0000e+00
Epoch 6/10
5/5 [==============================] - 5s 951ms/step - loss: 1.6215 - accuracy: 0.1776 - val_loss: 8.0844 - val_accuracy: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 5s 920ms/step - loss: 1.6215 - accuracy: 0.1382 - val_loss: 8.0531 - val_accuracy: 0.0

In [28]:
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


1/1 [==============================] - 0s 234ms/step


In [29]:
class_labels = train_generator.class_indices
class_labels = dict((v, k) for k, v in class_labels.items())


In [30]:
predicted_labels = [class_labels[prediction] for prediction in predicted_classes]
print(predicted_labels)


['Sweeton', 'Sweeton', 'Sweeton', 'Sweeton', 'Sweeton']
